# packages

In [1]:

from datetime import datetime
import matplotlib.pyplot as plt
import pandas as pd
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import MetaTrader5 as mt5
import pytz
import json

In [2]:
z = json.load(open('logins.json', 'r'))

# mt5 experimental portfolio

## connect to MT5

In [3]:
if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()

In [4]:

# request connection status and parameters
print(mt5.terminal_info())
# get data on MetaTrader 5 version
print(mt5.version())

TerminalInfo(community_account=False, community_connection=False, connected=True, dlls_allowed=False, trade_allowed=False, tradeapi_disabled=False, email_enabled=True, ftp_enabled=False, notifications_enabled=True, mqid=True, build=3550, maxbars=100000, codepage=0, ping_last=42646, community_balance=0.0, retransmission=0.0, company='MetaQuotes Software Corp.', name='MetaTrader 5', language='English', path='C:\\Program Files\\MetaTrader 5', ...)
(500, 3550, '16 Dec 2022')


In [5]:

# login to my account and print account info
account=z[0]['account_number']
server = z[0]['server']
password = z[0]['password']
authorized=mt5.login(login = account,server = server, password=password)
if authorized:
	# display trading account data in the form of a list
	print("Show account_info()._asdict():")
	account_info_dict = mt5.account_info()._asdict()
	for prop in account_info_dict:
		print("  {}={}".format(prop, account_info_dict[prop]))
else:
	print("failed to connect at account #{}, error code: {}".format(account, mt5.last_error()))

# get number of all financial instruments in the terminal
symbols_nr = mt5.symbols_total()
print(f'total symbols {symbols_nr}')


Show account_info()._asdict():
  login=30453044
  trade_mode=2
  leverage=1
  limit_orders=500
  margin_so_mode=0
  trade_allowed=True
  trade_expert=True
  margin_mode=0
  currency_digits=2
  fifo_close=False
  balance=3856.16
  credit=0.0
  profit=-943.12
  equity=2913.04
  margin=3761.84
  margin_free=94.32
  margin_level=77.43657359164663
  margin_so_call=10.0
  margin_so_so=0.0
  margin_initial=0.0
  margin_maintenance=0.0
  assets=0.0
  liabilities=0.0
  commission_blocked=0.0
  name=jaroslav klen
  server=AdmiralMarkets-Live
  currency=EUR
  company=Admiral Markets Group AS
total symbols 4979


In [6]:
# check the presence of active orders
orders=mt5.orders_total()
if orders>0:
    print("Total orders=",orders)
else:
    print("Orders not found")
 


Orders not found


In [7]:
# check the presence of open positions
positions_total=mt5.positions_total()
if positions_total>0:
    print("Total positions=",positions_total)
else:
    print("Positions not found")

Total positions= 4


In [8]:

positions=mt5.positions_get()
if positions==None:
    print("No positions, error code={}".format(mt5.last_error()))
elif len(positions)>0:
    print("Total positions =",len(positions))
    # display all open positions
    #for position in positions:
    #    print(position)

Total positions = 4


## open positions

In [9]:
# display these positions as a table using pandas.DataFrame
df_positions=pd.DataFrame(list(positions),columns=positions[0]._asdict().keys())
df_positions['time'] = pd.to_datetime(df_positions['time'], unit='s')
df_positions.drop(['time_update', 'time_msc', 'time_update_msc', 'external_id'], axis=1, inplace=True)
df_positions

,ticket,time,type,magic,identifier,reason,volume,price_open,sl,tp,price_current,swap,profit,symbol,comment
0,56148185,2021-04-27 17:14:35,0,0,56148185,0,22.0,75.606364,0.0,0.0,42.72,0.0,-677.66,DELL,
1,58873415,2021-06-07 14:42:53,0,0,58873415,0,1.0,283.550000,0.0,0.0,235.10,0.0,-48.45,EXS3,
2,59022414,2021-06-09 14:25:22,0,0,59022414,0,7.0,81.174286,0.0,0.0,78.98,0.0,-14.39,XSPU,
3,63252570,2021-08-16 11:33:36,0,0,63252570,0,51.0,31.012941,0.0,0.0,27.04,0.0,-202.62,AUCO,


In [10]:
# get the number of orders in history
from_date=datetime(2020,1,1)
to_date=datetime.now()
history_orders=mt5.history_orders_total(from_date, datetime.now())
if history_orders>0:
    print("Total history orders=",history_orders)
else:
    print("Orders not found in history")

Total history orders= 23


In [11]:
position_history_orders=mt5.history_orders_get(from_date, datetime.now())
if position_history_orders==None:
    #print("No orders with position #{}".format(position_id))
    print("error code =",mt5.last_error())
elif len(position_history_orders)>0:
    #print("Total history orders on position #{}: {}".format(position_id,len(position_history_orders)))
    # display all historical orders having a specified position ticket
    #for position_order in position_history_orders:        
    #    print(position_order)
    #print()
    # display these orders as a table using pandas.DataFrame
    df=pd.DataFrame(list(position_history_orders),columns=position_history_orders[0]._asdict().keys())
    df.drop(['time_expiration','type_time','state','position_by_id','reason','volume_current','price_stoplimit','sl','tp'], axis=1, inplace=True)
    df['time_setup'] = pd.to_datetime(df['time_setup'], unit='s')
    df['time_done'] = pd.to_datetime(df['time_done'], unit='s')

## history orders

In [12]:
df

,ticket,time_setup,time_setup_msc,time_done,time_done_msc,type,type_filling,magic,position_id,volume_initial,price_open,price_current,symbol,comment,external_id
0,56148185,2021-04-27 17:14:35,1619543675053,2021-04-27 17:14:35,1619543675562,0,2,0,56148185,10.0,0.0,100.50,DELL,,5008161953287514427
1,58873415,2021-06-07 14:42:53,1623076973427,2021-06-07 14:42:53,1623076973851,0,2,0,58873415,1.0,0.0,284.30,EXS3,,5008162306617364555
2,58933537,2021-06-08 12:00:48,1623153648819,2021-06-08 12:00:49,1623153649337,0,2,0,58933537,6.0,0.0,49.86,DBX1,,5008162314284848693
3,59022414,2021-06-09 14:25:21,1623248721359,2021-06-09 14:25:22,1623248722633,0,2,0,59022414,4.0,0.0,80.19,XSPU,,5008162323792119967
4,59344992,2021-06-15 15:52:43,1623772363175,2021-06-15 15:52:44,1623772364109,0,2,0,59344992,4.0,0.0,81.94,TNO,,5008162376156358211
5,59345295,2021-06-15 15:57:42,1623772662984,2021-06-15 15:57:44,1623772664160,0,2,0,59345295,25.0,0.0,12.83,RBOT,,5008162376186258754
6,61329464,2021-07-15 15:52:34,1626364354933,2021-07-15 15:52:35,1626364355240,0,2,0,61329464,15.0,0.0,23.12,ISPY,,5008162635355435861
7,61330848,2021-07-15 16:03:57,1626365037214,2021-07-15 16:03:58,1626365038145,0,2,0,61330848,16.0,0.0,21.75,XMCH,,5008162635423738747
8,63252215,2021-08-16 11:28:38,1629113318461,2021-08-16 11:28:40,1629113320067,0,2,0,63252215,2.0,0.0,153.98,MAAP,,5008162910251842711
9,63252570,2021-08-16 11:33:34,1629113614716,2021-08-16 11:33:36,1629113616140,0,2,0,63252570,11.0,0.0,27.08,AUCO,,5008162910281443393


In [13]:
position_history_orders=mt5.history_orders_get()


In [14]:
from_date=datetime(2020,1,1)
to_date=datetime.now()
deals=mt5.history_deals_total(from_date, to_date)
if deals>0:
    print("Total deals=",deals)
else:
    print("Deals not found in history")

Total deals= 37


In [15]:
# get deals for symbols whose names contain neither "EUR" nor "GBP"
deals = mt5.history_deals_get(from_date, to_date)
if deals == None:
    print("No deals, error code={}".format(mt5.last_error()))
elif len(deals) > 0:
    print("history_deals_get(from_date, to_date, group=\"*,!*EUR*,!*GBP*\") =", len(deals))
    # display all obtained deals 'as is'
    #for deal in deals:
    #    print("  ",deal)
    #print()
    # display these deals as a table using pandas.DataFrame
    df=pd.DataFrame(list(deals),columns=deals[0]._asdict().keys())
    df['time'] = pd.to_datetime(df['time'], unit='s')

history_deals_get(from_date, to_date, group="*,!*EUR*,!*GBP*") = 37


## history deals

In [16]:
df

,ticket,order,time,time_msc,type,entry,magic,position_id,reason,volume,price,commission,swap,profit,fee,symbol,comment,external_id
0,51096904,0,2021-04-27 13:09:27,1619528967353,2,0,0,0,0,0.0,0.00,0.00,0.0,1150.00,0.0,,Transfer from CW|2208917,
1,51121268,56148185,2021-04-27 17:14:35,1619543675562,0,0,0,56148185,0,10.0,100.52,-0.83,0.0,0.00,0.0,DELL,,5008161953287514427
2,53736750,0,2021-06-07 11:09:36,1623064176693,2,0,0,0,0,0.0,0.00,0.00,0.0,300.00,0.0,,Transfer from CW|2208917,
3,53750449,58873415,2021-06-07 14:42:53,1623076973851,0,0,0,58873415,0,1.0,283.55,-1.00,0.0,0.00,0.0,EXS3,,5008162306617364555
4,53807944,58933537,2021-06-08 12:00:49,1623153649337,0,0,0,58933537,0,6.0,49.84,-1.00,0.0,0.00,0.0,DBX1,,5008162314284848693
5,53893642,0,2021-06-09 14:23:06,1623248586830,2,0,0,0,0,0.0,0.00,0.00,0.0,300.00,0.0,,Transfer from CW|2208917,
6,53893800,59022414,2021-06-09 14:25:22,1623248722633,0,0,0,59022414,0,4.0,80.10,-0.82,0.0,0.00,0.0,XSPU,,5008162323792119967
7,54198694,0,2021-06-15 15:42:14,1623771734953,2,0,0,0,0,0.0,0.00,0.00,0.0,600.00,0.0,,Transfer from CW|2208917,
8,54199449,59344992,2021-06-15 15:52:44,1623772364109,0,0,0,59344992,0,4.0,81.87,-1.00,0.0,0.00,0.0,TNO,,5008162376156358211
9,54199738,59345295,2021-06-15 15:57:44,1623772664160,0,0,0,59345295,0,25.0,12.82,-0.83,0.0,0.00,0.0,RBOT,,5008162376186258754


## stats

In [17]:
df.profit.sum() # all transactions, including my inputs extraordinary payments and closed positions

3877.66

In [18]:
df.commission.sum() # fees paid on any transactions, positions open or closed

-21.5

In [19]:
df_positions.profit.sum() # current profit on open positions

-943.12

In [20]:
current_state = df.profit.sum() + df.commission.sum() + df_positions.profit.sum()
current_state # overal state if I would now close all positions

2913.04

In [21]:
my_inputs = df.loc[df['comment'] == 'Transfer from CW|2208917', 'profit'].sum()
my_inputs # my inputs

3550.0

In [22]:
profit = current_state - my_inputs
profit

-636.96

In [23]:
profit_perc = round(profit/my_inputs*100, 2)
f'{profit_perc} %'

'-17.94 %'

# bitcoins

In [24]:
btc_transactions = pd.read_csv('btc_transactions.csv')
btc_transactions

,datetime,amount EUR,fee EUR,total BTC
0,2022-02-24,100,1,0.002853


In [25]:
# login to my account and print account info
account=z[1]['account_number']
server = z[1]['server']
password = z[1]['password']
authorized=mt5.login(login = account,server = server, password=password)
if authorized:
	# display trading account data in the form of a list
	print("Show account_info()._asdict():")
	account_info_dict = mt5.account_info()._asdict()
	for prop in account_info_dict:
		print("  {}={}".format(prop, account_info_dict[prop]))
else:
	print("failed to connect at account #{}, error code: {}".format(account, mt5.last_error()))

# get number of all financial instruments in the terminal
symbols_nr = mt5.symbols_total()
print(f'total symbols {symbols_nr}')

failed to connect at account #41554358, error code: (-6, 'Terminal: Authorization failed')
total symbols 3773


In [26]:
tick = mt5.symbol_info_tick("BTCEUR")
tick

Tick(time=1647477568, bid=37151.81, ask=37264.09, last=0.0, volume=0, time_msc=1647477568930, flags=6, volume_real=0.0)

In [27]:
s = pd.Series(tick._asdict())
s['time'] = pd.to_datetime(s['time'], unit='s')
s

time           2022-03-17 00:39:28
bid                       37151.81
ask                       37264.09
last                           0.0
volume                         0.0
time_msc           1647477568930.0
flags                          6.0
volume_real                    0.0
dtype: object

## stats

In [28]:
current_state_btc = btc_transactions['total BTC'].sum() * s['bid']
current_state_btc

105.996755423691

In [29]:
profit_perc_btc = round((current_state_btc - btc_transactions['amount EUR'].sum())/btc_transactions['amount EUR'].sum() * 100, 2)
f'{profit_perc_btc} %'

'6.0 %'

In [30]:
# with open('logins.json', 'w') as fp:
    # json.dump(z, fp)